In [38]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_regression, RFE
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [25]:
df = pd.read_csv('Air_quality_index.csv')
df1 = df.copy()
df1.head(10)

,Latitude,Longitude,POC,State Name,County Name,City Name,Date Local,Month,Day,Event Type,Observation Count,Mean_Ozone_Concentration,Max_Ozone_Value,Hour_Max_Ozone_Concentration,Air_Quality_Index,AQI_Range
0,30.497478,-87.880258,1,Alabama,Baldwin,Fairhope,2022-02-28,2,28,None,1,0.038000,0.038,14.0,35,Good
1,30.497478,-87.880258,1,Alabama,Baldwin,Fairhope,2022-03-01,3,1,None,17,0.037235,0.054,12.0,50,Good
2,30.497478,-87.880258,1,Alabama,Baldwin,Fairhope,2022-03-02,3,2,None,17,0.038235,0.055,12.0,51,Moderate
3,30.497478,-87.880258,1,Alabama,Baldwin,Fairhope,2022-03-03,3,3,None,9,0.024333,0.043,14.0,40,Good
4,30.497478,-87.880258,1,Alabama,Baldwin,Fairhope,2022-03-04,3,4,None,17,0.049647,0.063,10.0,77,Moderate
5,30.497478,-87.880258,1,Alabama,Baldwin,Fairhope,2022-03-05,3,5,None,17,0.039353,0.044,9.0,41,Good
6,30.497478,-87.880258,1,Alabama,Baldwin,Fairhope,2022-03-06,3,6,None,17,0.029824,0.033,7.0,31,Good
7,30.497478,-87.880258,1,Alabama,Baldwin,Fairhope,2022-03-07,3,7,None,17,0.027706,0.029,9.0,27,Good
8,30.497478,-87.880258,1,Alabama,Baldwin,Fairhope,2022-03-08,3,8,None,17,0.018118,0.020,11.0,19,Good
9,30.497478,-87.880258,1,Alabama,Baldwin,Fairhope,2022-03-09,3,9,None,17,0.019235,0.022,12.0,20,Good


# Feature selection for Logistic Regression using filter method 

In [26]:
data = df1.copy()
data['AQI_Range'] = data.apply(lambda row: "High" if row['Air_Quality_Index'] > 100 else "Low", axis=1)
data.head(5)

df=data
df4=df
median_value = np.median(df['Air_Quality_Index'])
median_value

def AQI_Range(x):
    if x<=50.0:
        return "Good"
    elif x>50:
        return "Bad"
df4['AQI_Range'] = df4['Air_Quality_Index'] .apply(AQI_Range)

df4.head(5)

,Latitude,Longitude,POC,State Name,County Name,City Name,Date Local,Month,Day,Event Type,Observation Count,Mean_Ozone_Concentration,Max_Ozone_Value,Hour_Max_Ozone_Concentration,Air_Quality_Index,AQI_Range
0,30.497478,-87.880258,1,Alabama,Baldwin,Fairhope,2022-02-28,2,28,None,1,0.038000,0.038,14.0,35,Good
1,30.497478,-87.880258,1,Alabama,Baldwin,Fairhope,2022-03-01,3,1,None,17,0.037235,0.054,12.0,50,Good
2,30.497478,-87.880258,1,Alabama,Baldwin,Fairhope,2022-03-02,3,2,None,17,0.038235,0.055,12.0,51,Bad
3,30.497478,-87.880258,1,Alabama,Baldwin,Fairhope,2022-03-03,3,3,None,9,0.024333,0.043,14.0,40,Good
4,30.497478,-87.880258,1,Alabama,Baldwin,Fairhope,2022-03-04,3,4,None,17,0.049647,0.063,10.0,77,Bad


In [27]:
# Extract features and target variable
features = df4[['Mean_Ozone_Concentration', 'Max_Ozone_Value', 'Hour_Max_Ozone_Concentration']]
target = df4['AQI_Range']
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)


In [34]:
from sklearn.feature_selection import SelectKBest, f_classif

# (1) Feature selection for Logistic Regression using filter method (SelectKBest with f_classif)
selector = SelectKBest(score_func=f_classif, k=3)
X_train_filtered = selector.fit_transform(X_train, y_train)
X_test_filtered = selector.transform(X_test)

# Train Logistic Regression on the selected features
logreg = LogisticRegression()
logreg.fit(X_train_filtered, y_train)

# Predict on the test set
y_pred_logreg = logreg.predict(X_test_filtered)


In [35]:
acc_test = accuracy_score(y_test, y_pred_logreg)
print("Testing Accuracy:", round(acc_test, 2))
print(classification_report(y_test, y_pred_logreg))

Testing Accuracy: 0.93
              precision    recall  f1-score   support

         Bad       1.00      0.47      0.64      5209
        Good       0.93      1.00      0.96     34993

    accuracy                           0.93     40202
   macro avg       0.96      0.74      0.80     40202
weighted avg       0.94      0.93      0.92     40202



# Feature selection for Naive Bayes using wrapper method

In [44]:

clf = MultinomialNB()
rfe = RFE(clf, n_features_to_select=3)
X_train_rfe = rfe.fit_transform(X_train, y_train)
X_test_rfe = rfe.transform(X_test)

In [45]:
# Train Naive Bayes on the selected features
clf.fit(X_train_rfe, y_train)

# Predict on the test set
y_pred_clf = clf.predict(X_test_rfe)


In [46]:
# Evaluate the model
accuracy_clf = accuracy_score(y_test, y_pred_clf)
print(f'Naive Bayes Accuracy (with feature selection): {accuracy_clf}')
print(classification_report(y_test, y_pred_clf))


Naive Bayes Accuracy (with feature selection): 0.8704293318740361


/Users/nidhinambiar/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         Bad       0.00      0.00      0.00      5209
        Good       0.87      1.00      0.93     34993

    accuracy                           0.87     40202
   macro avg       0.44      0.50      0.47     40202
weighted avg       0.76      0.87      0.81     40202



/Users/nidhinambiar/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/nidhinambiar/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))




When using RFE with Naive Bayes, it's important to note that Naive Bayes is not inherently a feature selection-friendly algorithm because it assumes that all features are conditionally independent given the class label. This independence assumption may conflict with the feature ranking used by RFE.



# Feature selection for SVM using embedded method

In [53]:
features1 = ['Mean_Ozone_Concentration', 'Max_Ozone_Value', 'Hour_Max_Ozone_Concentration']
target1 = 'AQI_Range'

data1 = df1[features1 + [target1]]

# Convert categorical target variable to numerical values
data1[target1] = pd.Categorical(data1[target1])
data1['target_code'] = data1[target1].cat.codes

# Split the data into training and testing sets
X_train1, X_test1, y_train1, y_test1 = train_test_split(data1[features1], data1['target_code'], test_size=0.3, random_state=42)




/var/folders/nr/77158z8s5v39t2xq541m40_m0000gn/T/ipykernel_3101/1787470486.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1[target1] = pd.Categorical(data1[target1])
/var/folders/nr/77158z8s5v39t2xq541m40_m0000gn/T/ipykernel_3101/1787470486.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['target_code'] = data1[target1].cat.codes


In [54]:
# a. Training a linear SVM model
linear_svm_model = SVC(kernel='linear')
linear_svm_model.fit(X_train1, y_train1)

# Get feature importance using coefficients from SVM
feature_importance = linear_svm_model.coef_

# Print feature importance
print("Feature Importance:")
for feature, importance in zip(features1, feature_importance[0]):
    print(f"{feature}: {importance}")

# Make predictions on the test set
linear_svm_predictions = linear_svm_model.predict(X_test1)


Feature Importance:
Mean_Ozone_Concentration: -38.568210908400374
Max_Ozone_Value: -102.14165728921353
Hour_Max_Ozone_Concentration: 0.012944928894285113


In [55]:
# Evaluate the model
accuracy_svm = accuracy_score(y_test1, linear_svm_predictions)
print(f'SVM Accuracy (with feature selection): {accuracy_svm}')
print(classification_report(y_test1, linear_svm_predictions))

SVM Accuracy (with feature selection): 0.9446959521085186
              precision    recall  f1-score   support

           0       0.95      1.00      0.97     52500
           1       0.88      0.62      0.73      7194
           2       0.00      0.00      0.00       606
           3       0.00      0.00      0.00         3

    accuracy                           0.94     60303
   macro avg       0.46      0.41      0.43     60303
weighted avg       0.93      0.94      0.94     60303



/Users/nidhinambiar/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/nidhinambiar/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/nidhinambiar/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)